In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, PandasTools
from mordred import Calculator, descriptors
from Mold2_pywrapper import Mold2
from padelpy import from_smiles
from tqdm import tqdm
import warnings

In [2]:
warnings.filterwarnings('ignore')

# Defining Functions

In [3]:
def save_df(df, name):
    df.to_csv(f"../../data/negative_datasets/negative_dataset_with_descriptors/{name}.csv", index=False)

# Importing notebooks

In [4]:
pdb = pd.read_csv("../../data/negative_datasets/cleaned_datasets/pdb_cleaned.csv")
tox = pd.read_csv("../../data/negative_datasets/cleaned_datasets/tox21_cleaned.csv")
zinc = pd.read_csv("../../data/negative_datasets/cleaned_datasets/zinc15_cleaned.csv")

In [5]:
pdb.head()

,clean_smiles
0,CC1(C)O[C@H]2[C@@H]3OS(=O)(=O)O[C@@H]3CO[C@@]2...
1,CNC(=O)c1cccc(C)c1Nc1nc(N2CCN(c3ccccc3Cl)CC2)n...
2,CCNC(=O)Nc1cc2c(-c3ccnc(C)c3)ccc(C)c2cn1
3,Clc1ccccc1Nc1ccnc(Nc2ccc(-c3nnn[nH]3)cc2)n1
4,CC1=C(CCC(=O)O)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2...


In [6]:
tox.head()

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles,stripped_salt_smiles,neutralized_smiles,inchi,duplicated
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,InChI=1S/C9H10N2O3S2/c1-2-14-6-3-4-7-8(5-6)15-...,False
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O,CCN1C(=O)NC(c2ccccc2)C1=O,CCN1C(=O)NC(c2ccccc2)C1=O,InChI=1S/C11H12N2O2/c1-2-13-10(14)9(12-11(13)1...,False
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,InChI=1S/C20H32O/c1-3-20(21)13-11-18-17-9-8-14...,False
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,InChI=1S/C17H28N2O/c1-6-12-19(8-3)15(7-2)17(20...,False
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O,CC(O)(P(=O)(O)O)P(=O)(O)O,CC(O)(P(=O)(O)O)P(=O)(O)O,"InChI=1S/C2H8O7P2/c1-2(3,10(4,5)6)11(7,8)9/h3H...",True


In [7]:
zinc.head()

,clean_smiles
0,Cc1c(C(=O)Nc2ccccc2C(N)=O)nnn1-c1ccc2ncccc2c1
1,C[C@H]1CCCC[C@H]1NC(=O)NC(=O)CN1CCc2ccc([N+](=...
2,Brc1ccccc1-c1nc2c3ccccc3ncn2n1
3,CC[C@H](C)NC(=O)Nc1ccnn1C1CCCC1
4,CCn1c(SCc2cc(C(=O)OC)c(C)o2)nnc1-c1ccccc1OC


# Adding Mol Column

In [8]:
PandasTools.AddMoleculeColumnToFrame(pdb, smilesCol='clean_smiles', molCol='ROMol')
PandasTools.AddMoleculeColumnToFrame(tox, smilesCol='neutralized_smiles', molCol='ROMol')
PandasTools.AddMoleculeColumnToFrame(zinc, smilesCol='clean_smiles', molCol='ROMol')

# Calculate Parameters

## Define Function

In [9]:
def calculate_molecular_descriptors(df: pd.DataFrame, smiles_col: str, dataset_name: str):
    # save fucntion
    def save_df(df, name):
        df.to_csv(f"../../data/negative_datasets/negative_dataset_with_descriptors/{name}.csv", index=False)

    # adding mol column
    PandasTools.AddMoleculeColumnToFrame(df, smilesCol=smiles_col, molCol='ROMol')

    ##### RDKIT #####

    ##### RDKIT FINGERPRINTS #####

    print("Starting RDKit Fingerprints")

    # Function to generate RDKit fingerprints
    def generate_fingerprint_rdkit(smiles):
        mol = Chem.MolFromSmiles(smiles)
        fingerprint = Chem.RDKFingerprint(mol)
        return fingerprint

    # Function to generate the fingerprints and add each one to a new column
    df_rdkit_fingerprint = df.copy()
    fingerprints = df_rdkit_fingerprint[smiles_col].apply(generate_fingerprint_rdkit)
    num_bits = fingerprints.iloc[0].GetNumBits()

    for i in range(num_bits):
        col_name = f'Bit_{i}'
        df_rdkit_fingerprint[col_name] = fingerprints.apply(lambda x: x.GetBit(i))
    
    # save df
    save_df(df_rdkit_fingerprint, name=f"{dataset_name}_fingerprint_rdkit")

    print("Finished RDKit Fingerprints")
    
    ##### MORGAN FINGERPRINT #####

    print("Starting Morgan Fingerprints")

    # function to calculate morgan fingerprints
    def generate_fingerprint_morgan(smiles):
        mol = Chem.MolFromSmiles(smiles)
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2)  # Change radius as per your requirement
        return fingerprint
    
    df_morgan_fingerprint = df.copy()
    
    # calculate fingerprints and add each to a column
    # Generate fingerprints and create new columns for each bit
    fingerprints = df_morgan_fingerprint[smiles_col].apply(generate_fingerprint_morgan)
    num_bits = fingerprints.iloc[0].GetNumBits()

    for i in range(num_bits):
        col_name = f'Bit_{i}'
        df_morgan_fingerprint[col_name] = fingerprints.apply(lambda x: x.GetBit(i))
    
    # save df
    save_df(df_morgan_fingerprint, name=f"{dataset_name}_fingerprint_morgan")

    print("Finished Morgan Fingerprints")

    ##### RDKIT DESCRIPTORS #####

    print("Starting RDKit Descriptors")

    # function to calculate descriptors
    def calculate_descriptors(smiles):
        mol = Chem.MolFromSmiles(smiles)

        descriptors = {}
        for descriptor_name, descriptor_function in Descriptors.descList:
            try:
                descriptors[descriptor_name] = descriptor_function(mol)
            except:
                descriptors[descriptor_name] = None

        return pd.Series(descriptors)

    # calculating the descriptors
    df_rdkit_descriptors = df.copy()
    rdkit_descriptors = df_rdkit_descriptors[smiles_col].apply(calculate_descriptors)
    df_rdkit_descriptors = pd.concat([df_rdkit_descriptors, rdkit_descriptors], axis=1)

    # save dataset
    save_df(df_rdkit_descriptors, name=f"{dataset_name}_rdkit")

    print("Finished RDKit Descriptors")

    ##### MORDRED #####

    print("Starting Mordred")

    df_mordred = df.copy()

    # initializing object for mordred descriptors calculation, for only 2d descriptors, and both 2d and 3d descriptors
    calc_2d = Calculator(descriptors, ignore_3D=True)
    calc_2d_3d = Calculator(descriptors, ignore_3D=False)

    # calculating descriptors
    mordred_descriptors_2d = calc_2d.pandas(df_mordred["ROMol"])
    mordred_descriptors_2d_3d = calc_2d_3d.pandas(df_mordred["ROMol"])

    # concating to original dataset
    df_mordred_2d = pd.concat([df_mordred, mordred_descriptors_2d], axis=1)
    df_mordred_2d_3d = pd.concat([df_mordred, mordred_descriptors_2d_3d], axis=1)

    # saving datasets
    save_df(df_mordred_2d, name=f"{dataset_name}_mordred_2d")
    save_df(df_mordred_2d_3d, name=f"{dataset_name}_mordred_2d_and_3d")

    print("Finished Mordred")

    ##### MOLD 2 #####

    print("Starting Mold 2")

    df_mold2 = df.copy()

    # initializing object
    mold2 = Mold2()

    # calculating descriptors
    mold2_descriptors = mold2.calculate(df_mold2["ROMol"])

    # concat
    df_mold2 = pd.concat([df_mold2, mold2_descriptors], axis=1)

    # save df
    save_df(df_mold2, name=f"{dataset_name}_mold2")

    print("Finished Mold 2")

    ##### PADEL #####

    print("Starting Padel")

    df_padel = df.copy()

    # smiles list
    smiles = df_padel[smiles_col].tolist()

    # calculating the descriptors and saving to a list
    # molecules with descriptors that can't be calculated are saved in the list error
    correct_descriptors = []
    error = []
    for smi in tqdm(smiles):
        try:
            desc = from_smiles(smi)
            desc[smiles_col] = smi
            correct_descriptors.append(desc)
        except:
            error.append(smi)

    # adding molecules to dataframe
    descriptors_df = pd.DataFrame.from_records(correct_descriptors)
    df_padel = pd.merge(df_padel, descriptors_df, on=smiles_col, how="outer")

    # save df
    save_df(df_padel, name=f"{dataset_name}_padel")

    print("Finished Padel")
    

## Run Calculation

In [46]:
calculate_molecular_descriptors(
    df=pdb,
    smiles_col="clean_smiles",
    dataset_name="pdb"
)

Starting RDKit Fingerprints
Finished RDKit Fingerprints
Starting Morgan Fingerprints
Finished Morgan Fingerprints
Starting RDKit Descriptors
Finished RDKit Descriptors
Starting Mordred


  2%|▏         | 247/12246 [00:38<19:17, 10.36it/s] 

[18:17:45] Explicit valence for atom # 29 C greater than permitted


  5%|▍         | 563/12246 [01:23<43:43,  4.45it/s]  

[18:18:28] Explicit valence for atom # 20 C greater than permitted


...

[19:52:06] Explicit valence for atom # 26 C greater than permitted


100%|██████████| 12246/12246 [1:00:28<00:00,  3.37it/s]


Finished Mordred
Starting Mold 2
Mold2 calculates a large and diverse set of molecular descriptors encoding two-
dimensional chemical structure information. Comparative analysis of Mold2 descriptors
with those calculated from commercial software on several published datasets
demonstrated that Mold2 descriptors convey sufficient structural information. In addition,
better models were generated using Mold2 descriptors than the compared commercial
software packages. This publicly available software is developed by the Center for
Bioinformatics, which is led by Dr. Weida Tong, at the National Center for Toxicological
Research (NCTR).
    
Mold2 is a product designed and produced by the National Center for Toxicological
Research (NCTR).  FDA and NCTR retain ownership of this product.

Please address any questions or suggestions to Dr. Huixiao Hong, National Center for Toxicological
Research, at 870-543-7296 or Huixiao.Hong@fda.hhs.gov.

###################################

Should you publis

100%|██████████| 12246/12246 [38:17:37<00:00, 11.26s/it]   


Finished Padel


In [10]:
calculate_molecular_descriptors(
    df=tox,
    smiles_col="neutralized_smiles",
    dataset_name="tox"
)

Starting RDKit Fingerprints
Finished RDKit Fingerprints
Starting Morgan Fingerprints
Finished Morgan Fingerprints
Starting RDKit Descriptors
Finished RDKit Descriptors
Starting Mordred


100%|██████████| 7506/7506 [08:33<00:00, 14.62it/s]  


Finished Mordred
Starting Mold 2
Mold2 calculates a large and diverse set of molecular descriptors encoding two-
dimensional chemical structure information. Comparative analysis of Mold2 descriptors
with those calculated from commercial software on several published datasets
demonstrated that Mold2 descriptors convey sufficient structural information. In addition,
better models were generated using Mold2 descriptors than the compared commercial
software packages. This publicly available software is developed by the Center for
Bioinformatics, which is led by Dr. Weida Tong, at the National Center for Toxicological
Research (NCTR).
    
Mold2 is a product designed and produced by the National Center for Toxicological
Research (NCTR).  FDA and NCTR retain ownership of this product.

Please address any questions or suggestions to Dr. Huixiao Hong, National Center for Toxicological
Research, at 870-543-7296 or Huixiao.Hong@fda.hhs.gov.

###################################

Should you publis

100%|██████████| 7506/7506 [13:54:10<00:00,  6.67s/it]   


Finished Padel


In [11]:
calculate_molecular_descriptors(
    df=zinc,
    smiles_col="clean_smiles",
    dataset_name="zinc"
)

Starting RDKit Fingerprints
Finished RDKit Fingerprints
Starting Morgan Fingerprints
Finished Morgan Fingerprints
Starting RDKit Descriptors
Finished RDKit Descriptors
Starting Mordred


100%|██████████| 9971/9971 [13:40<00:00, 12.15it/s]   


Finished Mordred
Starting Mold 2
Mold2 calculates a large and diverse set of molecular descriptors encoding two-
dimensional chemical structure information. Comparative analysis of Mold2 descriptors
with those calculated from commercial software on several published datasets
demonstrated that Mold2 descriptors convey sufficient structural information. In addition,
better models were generated using Mold2 descriptors than the compared commercial
software packages. This publicly available software is developed by the Center for
Bioinformatics, which is led by Dr. Weida Tong, at the National Center for Toxicological
Research (NCTR).
    
Mold2 is a product designed and produced by the National Center for Toxicological
Research (NCTR).  FDA and NCTR retain ownership of this product.

Please address any questions or suggestions to Dr. Huixiao Hong, National Center for Toxicological
Research, at 870-543-7296 or Huixiao.Hong@fda.hhs.gov.

###################################

Should you publis

100%|██████████| 9971/9971 [15:40:24<00:00,  5.66s/it]   


Finished Padel
